In [1]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.reuters.com/robots.txt")
rp.read()

In [2]:
# 특정 User-agent가 url에 접근 가능한지 확인한다.
rp.can_fetch(useragent="*", url = "https://reuters.com/sitemap.xml")

True

In [3]:
sitemaps = rp.site_maps()
sitemaps

['https://www.reuters.com/arc/outboundfeeds/sitemap-index/?outputType=xml',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap-index/?outputType=xml',
 'https://www.reuters.com/sitemap_video_index.xml',
 'https://www.reuters.com/brandfeature/sitemap']

In [4]:
# !pip install xmltodict
# xmltodict 패키지 설치

In [5]:
import xmltodict
import requests

url = sitemaps[0]
sitemaps = xmltodict.parse(requests.get(url).text)

In [6]:
sitemaps

{'sitemapindex': {'@xmlns': 'http://www.sitemaps.org/schemas/sitemap/0.9',
  'sitemap': [{'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml',
    'lastmod': '2023-02-28T06:38:40.936Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=100',
    'lastmod': '2023-02-28T06:38:40.936Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=200',
    'lastmod': '2023-02-28T06:38:40.936Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=300',
    'lastmod': '2023-02-28T06:38:40.936Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=400',
    'lastmod': '2023-02-28T06:38:40.936Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=500',
    'lastmod': '2023-02-28T06:38:40.936Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=600',
    'lastmod': '2023-02-28T06:38:40.

In [7]:
# sitemap url을 확보
sitemap_urls = [sitemap['loc'] for sitemap in sitemaps['sitemapindex']['sitemap']]

# 개수 파악
print(f"{len(sitemap_urls)}개의 사이트맵 URL이 있습니다.")

# 샘플로 앞의 sitemap url 5개만 확인
sitemap_urls[:5]

# display(sitemap_urls)를 하면 한줄에 요소 하나씩 보여준다.

96개의 사이트맵 URL이 있습니다.


['https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=100',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=200',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=300',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=400']

In [8]:
# 사이트맵을 활용한 웹 크롤러 작성하기
# 걔 중 제일 첫번째 sitemap_urls에 대해서 조회
url = sitemap_urls[0]
news_dict = xmltodict.parse(requests.get(url).text)
news_dict

{'urlset': {'@xmlns': 'http://www.sitemaps.org/schemas/sitemap/0.9',
  '@xmlns:image': 'http://www.google.com/schemas/sitemap-image/1.1',
  'url': [{'loc': 'https://www.reuters.com/markets/europe/global-markets-view-europe-2023-02-28/',
    'lastmod': '2023-02-28T06:30:51.395Z',
    'image:image': {'image:loc': 'https://www.reuters.com/resizer/VAhiE2wIqGF42htLlOrAm5QKpbA=/cloudfront-us-east-2.images.arcpublishing.com/reuters/HJX63HRYZNJLJGAQPOSEHQV45M.jpg',
     'image:caption': 'The German share price index DAX graph is pictured at the stock exchange in Frankfurt, Germany, February 27, 2023.    REUTERS/Staff'}},
   {'loc': 'https://www.reuters.com/business/retail-consumer/home-zara-fast-slow-fashion-collide-2023-02-28/',
    'lastmod': '2023-02-28T06:29:10.080Z',
    'image:image': {'image:loc': 'https://www.reuters.com/resizer/bJSj22xxE8PlHfv9BLCLGhzVVyw=/cloudfront-us-east-2.images.arcpublishing.com/reuters/E4WVOANN2FJD3GHH354WDLWAJY.jpg',
     'image:caption': 'Marian Fernandez, 56

In [9]:
news_urls = [news['loc'] for news in news_dict['urlset']['url']]
news_urls

['https://www.reuters.com/markets/europe/global-markets-view-europe-2023-02-28/',
 'https://www.reuters.com/business/retail-consumer/home-zara-fast-slow-fashion-collide-2023-02-28/',
 'https://www.reuters.com/lifestyle/sports/england-captain-stokes-has-no-regrets-over-follow-on-2023-02-28/',
 'https://www.reuters.com/markets/asia/indias-adani-plans-repay-up-790-mln-share-backed-loans-by-march-sources-2023-02-28/',
 'https://www.reuters.com/lifestyle/sports/smith-says-australia-batsmen-will-ditch-risky-tempo-third-test-2023-02-28/',
 'https://www.reuters.com/technology/cryptoverse-bitcoin-moves-towards-satoshis-payment-dream-2023-02-28/',
 'https://www.reuters.com/markets/europe/ecb-has-started-win-inflation-fight-lane-says-2023-02-28/',
 'https://www.reuters.com/markets/rates-bonds/banks-pile-into-euro-zone-bond-sales-rates-shoot-up-2023-02-28/',
 'https://www.reuters.com/business/adecco-sees-signs-hiring-slowdown-start-2023-2023-02-28/',
 'https://www.reuters.com/world/us/tennessee-me

In [10]:
# URL을 활용하여 데이터 다운로드하기

In [11]:
%%time
# session 추상화
session = requests.Session()

# 앞의 5개 링크만 불러온다.
for url in news_urls[:5]:
    file = url.split("/")[-2] + ".html"
    
    response = session.get(url)
    if response.ok:
        with open("datas/"+file, "w+b") as f:
            f.write(response.text.encode("utf-8"))
    else:
        print(f"error with URL : {url}")

Wall time: 923 ms


In [12]:
# API가 사용 가능하다면 API가 가장 좋음

# 웹 크롤링이나 스크래핑 | RPA를 쓰는 경우
# 왜 쓰는지 알고 써야함